# Read a MySQL DataBase and generate specific output

In [5]:
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(row):
    latexfile=r'''
%'''+row.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+row.preamble+r'''
'''+row.address+r'''
\signature{'''+row.sign+r'''}
\begin{document}
\begin{letter}{'''+row.para+r'''}

\opening{'''+row.opening+r'''} 

'''+row.body+r'''

\closing{'''+row.closing+r'''}
\end{letter}
\end{document}
'''
    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        pdf='carta_{:d}.pdf'.format(row['id'])
        dwld_dir='downloads'
        kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        sys.exit('LaTeX ERROR:')    
    return latexfile

def db_to_pdf(choose_db=0):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[0])

    df=pd.read_sql('select * from cartas;', con=conn)

    sg=pd.read_sql('select * from cartas_sign;', con=conn)
    
    
    conn.close()
    #print(df.shape[0])
    #return df,sg
    if df.shape[0]:
        c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
        c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']
    else:
        return df

    
    
    return generate_pdf(c)

In [ ]:
latexfile=db_to_pdf()

In [17]:
df,sg=latexfile

In [20]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]

In [21]:
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [ ]:
generate_pdf(c)

## Manual

In [7]:
password=getpass.getpass()

········


In [8]:
tipo_carta=['Diego_Restrepo', 'cartas_gfif']
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[0])

df=pd.read_sql('select * from cartas;', con=conn)

sg=pd.read_sql('select * from cartas_sign;', con=conn)

In [9]:
conn.close()

Last entry

In [14]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [ ]:
c

In [ ]:
c.sign

## Search in body

In [50]:
id=99
if id >0:
    dfs=df[df.id==id].reset_index(drop=True)
else:    
    dfs=df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')].reset_index(drop=True)

In [ ]:
dfs

In [52]:
c=dfs.loc[dfs.index.values[-1]]

In [ ]:
c

In [ ]:
c.keys().values

In [ ]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''

In [56]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

In [ ]:
print(latexfile)

In [58]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [59]:
f.write(latexfile)

2017

In [60]:
f.name

'/tmp/tmppt2pwnu1.tex'

In [61]:
f.close()

In [62]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [ ]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    